# Import Modules

In [1]:
import os
import sys
import numpy as np
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
from tqdm.notebook import tqdm
from collections import namedtuple
from collections import defaultdict
from pprint import pprint

# Configure display options
pd.set_option('display.max_columns', 200)
pd.set_option('display.max_rows', 200)
%config InlineBackend.figure_format = 'retina'

# Add project directory to sys.path
pdir = os.path.abspath(os.path.join(os.getcwd(), '.'))
sys.path.insert(1, pdir)
# from myutils import *

# Other module imports
import ast, math, swifter, csv, json, itertools as it, portion as P

# Set plot style
# plt.style.use('ggplot')

In [2]:
def set_data(df, mode='pcap', tz=0):
    def nr_serv_cel(row):
        pos = row.serv_cel_pos
        if pos == 255:
            return 65535, -160, -50
        else:
            return row[f'PCI{pos}'], row[f'RSRP{pos}'], row[f'RSRQ{pos}']
    
    if mode == 'pcap':
        common_column_names = [
            'seq', 'rpkg', 'frame_id', 'Timestamp', 'Timestamp_epoth', 'lost', 'excl', 'latency',
            'xmit_time', 'xmit_time_epoch', 'arr_time', 'arr_time_epoch',
        ]
        
        if df.empty:
            return pd.DataFrame(columns=common_column_names)
        
        date_columns = ['Timestamp', 'xmit_time', 'arr_time']
        df[date_columns] = df[date_columns].apply(pd.to_datetime)
        df[['seq', 'rpkg', 'frame_id']] = df[['seq', 'rpkg', 'frame_id']].astype('Int32')
        df[['Timestamp_epoch', 'xmit_time_epoch', 'arr_time_epoch', 'latency']] = \
            df[['Timestamp_epoch', 'xmit_time_epoch', 'arr_time_epoch', 'latency']].astype('float32')
        df[['lost', 'excl']] = df[['lost', 'excl']].astype('boolean')

    if mode in ['lte', 'nr']:
        common_column_names = [
            'Timestamp', 'type_id', 'PCI', 'RSRP', 'RSRQ', 'serv_cel_index', 'EARFCN', 'NR_ARFCN', 
            'num_cels', 'num_neigh_cels', 'serv_cel_pos', 'PCI0', 'RSRP0', 'RSRQ0',
        ]
        
        if df.empty:
            return pd.DataFrame(columns=common_column_names)
        
        if mode == 'lte':
            columns_mapping = {
                'RSRP(dBm)': 'RSRP',
                'RSRQ(dB)': 'RSRQ',
                'Serving Cell Index': 'serv_cel_index',
                'Number of Neighbor Cells': 'num_neigh_cels',
                'Number of Detected Cells': 'num_cels',
            }
            columns_order = [*common_column_names, *df.columns[df.columns.get_loc('PCI1'):].tolist()]
            
            df = df.rename(columns=columns_mapping).reindex(columns_order, axis=1)
            df['serv_cel_index'] = np.where(df['serv_cel_index'] == '(MI)Unknown', '3_SCell', df['serv_cel_index'])
            df['num_cels'] = df['num_neigh_cels'] + 1

        if mode == 'nr':
            columns_mapping = {
                'Raster ARFCN': 'NR_ARFCN',
                'Serving Cell Index': 'serv_cel_pos',
                'Num Cells': 'num_cels',
            }
            columns_order = [*common_column_names, *df.columns[df.columns.get_loc('PCI1'):].tolist()]
            
            df = df.rename(columns=columns_mapping).reindex(columns_order, axis=1)
            df[['PCI', 'RSRP', 'RSRQ']] = df.apply(nr_serv_cel, axis=1, result_type='expand')
            df['serv_cel_index'] = np.where(df['serv_cel_pos'] == 255, df['serv_cel_index'], 'PSCell')
            df['num_neigh_cels'] = np.where(df['serv_cel_pos'] == 255, df['num_cels'], df['num_cels'] - 1)
        
        df['Timestamp'] = pd.to_datetime(df['Timestamp']) + pd.Timedelta(hours=tz)
        df[['type_id', 'serv_cel_index']] = df[['type_id', 'serv_cel_index']].astype('category')
        df[['EARFCN', 'NR_ARFCN']] = df[['EARFCN', 'NR_ARFCN']].astype('Int32')
        df[['num_cels', 'num_neigh_cels', 'serv_cel_pos']] = df[['num_cels', 'num_neigh_cels', 'serv_cel_pos']].astype('UInt8')

        for tag in df.columns:
            if tag.startswith('PCI'):
                df[tag] = df[tag].astype('Int32')
            if tag.startswith(('RSRP', 'RSRQ')):
                df[tag] = df[tag].astype('float32')

    return df

In [3]:
##### Sheng-Ru Latest Version (0925)

def parse_mi_ho(df, tz=8):

    # df = pd.read_csv(f)
    df["Timestamp"] = df["Timestamp"].swifter.apply(lambda x: pd.to_datetime(x) + dt.timedelta(hours=tz))
    nr_pci = 'O'
    scells = []

    def NR_OTA(idx):

        if df["type_id"].iloc[idx] == "5G_NR_RRC_OTA_Packet": return True
        else: return False
    
    def LTE_SERV_INFO(idx):

        if df["type_id"].iloc[idx] == "LTE_RRC_Serv_Cell_Info": return True
        else: return False
    

    def find_1st_after(start_idx, target, look_after=1):
        for j in range(start_idx, len(df)):
            t_ = df["Timestamp"].iloc[j]
            if NR_OTA(j) or LTE_SERV_INFO(j):
                continue
            if (t_ - t).total_seconds() > look_after:
                return None, None
            if df[target].iloc[j] not in [0,'0'] and not np.isnan(df[target].iloc[j]):
                return t_, j
        return None, None
    
    def find_1st_before(start_idx, target, look_before=1):
        for j in range(start_idx, -1, -1):
            t_ = df["Timestamp"].iloc[j]
            if NR_OTA(j) or LTE_SERV_INFO(j):
                continue
            if (t - t_).total_seconds() > look_before:
                return None, None
            if df[target].iloc[j] not in [0,'0'] and not np.isnan(df[target].iloc[j]):
                return t_, j
        return None, None
    
    def find_1st_before_with_special_value(start_idx, target, target_value, look_before=1):
        for j in range(start_idx, -1, -1):
            t_ = df["Timestamp"].iloc[j]
            if NR_OTA(j) or LTE_SERV_INFO(j):
                continue
            if (t - t_).total_seconds() > look_before:
                return None, None
            if df[target].iloc[j] in [target_value] and not np.isnan(df[target].iloc[j]):
                return t_, j
        return None, None
    
    def find_in_D_exact(targets):

        l = []
        # In l : (second, ho_type)
        for target in targets:
            for ho in D[target]:
                l.append(((t - ho.start).total_seconds(), target))

        if len(l) != 0:
            for x in l:
                if (x[0]== 0):
                    return x[1]
        
        return None
    
    def find_in_D_first_before(targets, look_before=1):

        l = []
        # In l : (second, ho_type)
        for target in targets:
            for ho in D[target]:
                l.append(((t - ho.end).total_seconds(), target, ho))

        if len(l) != 0:
            closest = min(filter(lambda x: x[0] > 0, l), key=lambda x: x[0])
            if 0 <= closest[0] < look_before:
                return closest[1], closest[2]
        
        return None, None
    
    HO = namedtuple('HO',['start', 'end', 'others', 'trans'], defaults=[None,None,'',''])
    
    D = {
        'Conn_Rel':[], 
        'Conn_Req':[], # Setup
        'LTE_HO': [], # LTE -> newLTE
        'MN_HO': [], # LTE + NR -> newLTE + NR
        'MN_HO_to_eNB': [], # LTE + NR -> newLTE
        'SN_setup': [], # LTE -> LTE + NR => NR setup
        'SN_Rel': [], # LTE + NR -> LTE
        'SN_HO': [], # LTE + NR -> LTE + newNR  
        'RLF_II': [],
        'RLF_III': [],
        'SCG_RLF': [],
        'Add_SCell': [],
        }

    for i in range(len(df)):

        # Pass NR RRC packet. In NSA mode, LTE RRC packet include NR packet message.
        if NR_OTA(i) or LTE_SERV_INFO(i):
            continue

        others = ''
        t = df["Timestamp"].iloc[i]

        if df["rrcConnectionRelease"].iloc[i] == 1:
            D['Conn_Rel'].append(HO(start=t))
            nr_pci = 'O'

        if df["rrcConnectionRequest"].iloc[i] == 1:
            
            # Define end of rrcConnectionRequest to be rrcConnectionReconfigurationComplete or securityModeComplete.
            a = find_1st_after(i, 'rrcConnectionReconfigurationComplete',look_after=2)[0]
            b = find_1st_after(i, 'securityModeComplete',look_after=2)[0]
            if a is None and b is None: end = None
            elif a is None and b is not None: end = b
            elif a is not None and b is None: end = a 
            else: end = a if a > b else b
            
            serv_cell, serv_freq = df["PCI"].iloc[i], int(df["Freq"].iloc[i])
            trans = f'? -> ({serv_cell}, {serv_freq})'
            D['Conn_Req'].append(HO(start=t,end=end,trans=trans))
            nr_pci = 'O'
        
        if df["lte-rrc.t304"].iloc[i] == 1:
            
            end, _ = find_1st_after(i, 'rrcConnectionReconfigurationComplete')
            serv_cell, target_cell = df["PCI"].iloc[i], int(df['lte_targetPhysCellId'].iloc[i])
            serv_freq, target_freq = int(df["Freq"].iloc[i]), int(df['dl-CarrierFreq'].iloc[i])

            if df["SCellToAddMod-r10"].iloc[i] == 1:
                n =len(str(df["SCellIndex-r10.1"].iloc[i]).split('@'))
                others += f' Set up {n} SCell.'
            else:
                scells = []
            
            if serv_freq != target_freq:
                a,b = find_1st_before(i, "rrcConnectionReestablishmentRequest", 1)
                others += " Inter frequency HO."
                if a is not None:
                    others += " Near after RLF."
                
            if df["nr-rrc.t304"].iloc[i] == 1 and df["dualConnectivityPHR: setup (1)"].iloc[i] == 1:
                
                if serv_cell == target_cell and serv_freq == target_freq:

                    a, _ = find_1st_before(i, "rrcConnectionReestablishmentRequest", 2)
                    
                    if a is not None:

                        ho_type, ho = find_in_D_first_before(['RLF_II', 'RLF_III'], 2)
                        others += f' Near after RLF of trans: {ho.trans}.'

                    else:
                        
                        ho_type, _ = find_in_D_first_before(['MN_HO_to_eNB', 'SN_Rel'], 2)
                        if ho_type is not None:
                            others += f' Near after {ho_type}.'

                    ori_serv = nr_pci
                    nr_pci = int(df['nr_physCellId'].iloc[i])
                    trans = f'({serv_cell}, {serv_freq}) | {ori_serv} -> {nr_pci}'
                    D['SN_setup'].append(HO(start=t, end=end, others=others, trans=trans))

                else:
                    
                    nr_pci = int(df['nr_physCellId'].iloc[i])
                    trans = f'({serv_cell}, {serv_freq}) -> ({target_cell}, {target_freq}) | {nr_pci}'
                    D['MN_HO'].append(HO(start=t, end=end, others=others, trans=trans))

            else:
                
                if serv_cell == target_cell and serv_freq == target_freq:

                    a, b = find_1st_before(i, "scgFailureInformationNR-r15")
                    if a is not None:
                        others += " Caused by scg-failure."
                    
                    orig_serv = nr_pci
                    nr_pci = 'O'
                    trans = f'({serv_cell}, {serv_freq}) | {orig_serv} -> {nr_pci}'
                    D['SN_Rel'].append(HO(start=t, end=end, others=others, trans=trans))
                    
                else:

                    a, _ = find_1st_before(i,"rrcConnectionSetup",3)
                    if a is not None:
                        others += ' Near After connection setup.'
                    if nr_pci == 'O':
                        trans = f'({serv_cell}, {serv_freq}) -> ({target_cell}, {target_freq}) | {nr_pci}'
                        D['LTE_HO'].append(HO(start=t, end=end, others=others, trans=trans))
                    else:
                        orig_serv = nr_pci
                        nr_pci = 'O'
                        trans = f'({serv_cell}, {serv_freq}) -> ({target_cell}, {target_freq}) | {orig_serv} -> {nr_pci}'
                        D['MN_HO_to_eNB'].append(HO(start=t, end=end, others=others, trans=trans))


        if df["nr-rrc.t304"].iloc[i] == 1 and not df["dualConnectivityPHR: setup (1)"].iloc[i] == 1:

            end, _ = find_1st_after(i,'rrcConnectionReconfigurationComplete')
        
            serv_cell, serv_freq = df["PCI"].iloc[i], int(df["Freq"].iloc[i])
            orig_serv = nr_pci
            nr_pci = int(df['nr_physCellId'].iloc[i])
            trans = f'({serv_cell}, {serv_freq}) | {orig_serv} -> {nr_pci}'
            D['SN_HO'].append(HO(start=t,end=end,trans=trans))


        if df["rrcConnectionReestablishmentRequest"].iloc[i] == 1:

            end1, _ = find_1st_after(i, 'rrcConnectionReestablishmentComplete', look_after=1)
            b, _ = find_1st_after(i, 'rrcConnectionReestablishmentReject', look_after=1)
            end2, _ = find_1st_after(i, 'securityModeComplete',look_after=3)

            others += ' ' + df["reestablishmentCause"].iloc[i] + '.'
            scells = []

            c, _ = find_1st_before(i, 'scgFailureInformationNR-r15', 1)
            if c != None:
                others  += ' caused by scgfailure.'
                
            serv_cell, rlf_cell = df["PCI"].iloc[i], int(df['physCellId.3'].iloc[i])
            serv_freq = int(df['Freq'].iloc[i])
            
            # Type II & Type III
            if end1 is not None: 

                orig_serv = nr_pci
                nr_pci = 'O'
                _, idx = find_1st_before_with_special_value(i, 'PCI', rlf_cell, look_before=10)
                rlf_freq = int(df['Freq'].iloc[idx])
                trans = f'({rlf_cell}, {rlf_freq}) -> ({serv_cell}, {serv_freq}) | {orig_serv} -> {nr_pci}'
                D['RLF_II'].append(HO(start=t,end=end1,others=others,trans=trans))

            elif b is not None and end2 is not None:
                
                orig_serv = nr_pci
                nr_pci = 'O'
                _, idx = find_1st_before_with_special_value(i, 'PCI', rlf_cell, look_before=10)
                rlf_freq = int(df['Freq'].iloc[idx])

                _, idx = find_1st_after(i, "rrcConnectionRequest", 2)
                recon_cell, recon_freq = df['PCI'].iloc[idx], int(float(df['Freq'].iloc[idx]))
                
                trans = f'({rlf_cell}, {rlf_freq}) -> ({recon_cell}, {recon_freq}) | {orig_serv} -> {nr_pci}'
                D['RLF_III'].append(HO(start=t,end=end2,others=others,trans=trans))
                
            else:
                others+=' No end.'
                D['RLF_II'].append(HO(start=t,others=others))
                print('No end for RLF')

        if df["scgFailureInformationNR-r15"].iloc[i] == 1:

            others += ' ' + df["failureType-r15"].iloc[i] + '.'
            a, idx1 = find_1st_after(i, "rrcConnectionReestablishmentRequest", look_after=1)
            b, idx2 = find_1st_after(i, "lte-rrc.t304", look_after=10)

            if a is not None:

                end1, _ = find_1st_after(idx1, 'rrcConnectionReestablishmentComplete', look_after=1)
                b, _ = find_1st_after(idx1, 'rrcConnectionReestablishmentReject', look_after=1)
                end2 = find_1st_after(idx1, 'securityModeComplete',look_after=3)[0]

                others += ' Result in rrcReestablishment.'
                    
                # Type II & Type III Result
                if end1 is not None: 
                    D['SCG_RLF'].append(HO(start=t,end=end1,others=others))
                elif b is not None and end2 is not None: 
                    D['SCG_RLF'].append(HO(start=t,end=end2,others=others))
                else:
                    others += ' No end.'
                    D['SCG_RLF'].append(HO(start=t,others=others))
                    print('No end for scg failure result in rrcReestablishment.')

            elif b is not None:

                end, _ = find_1st_after(idx2, 'rrcConnectionReconfigurationComplete')
                serv_cell, target_cell = df["PCI"].iloc[idx2], df['lte_targetPhysCellId'].iloc[idx2]
                serv_freq, target_freq = int(df["Freq"].iloc[idx2]), df['dl-CarrierFreq'].iloc[idx2]
                others += ' Result in gNB release.'
                # We do not change nr_pci here. Instead, we will change it at gNB_Rel event.
                trans = f'({serv_cell}, {serv_freq}) | {nr_pci} -> O'
                
                if serv_cell == target_cell and serv_freq == target_freq:
                    D['SCG_RLF'].append(HO(start=t,end=end,others=others,trans=trans))
                else:
                    others += ' Weird gNB release.'
                    print('Weird for scg failure result in gNb Release.')
                    D['SCG_RLF'].append(HO(start=t,end=end,others=others,trans=trans))                  

            else:

                print('No end for scg failure.')
                others += ' No end.'
                D['SCG_RLF'].append(HO(start=t,others=others))
        
        if df['SCellToAddMod-r10'].iloc[i] == 1 and df['physCellId-r10'].iloc[i] != 'nr or cqi report':

            others = ''
            pcis = str(df["physCellId-r10"].iloc[i]).split('@')
            freqs = str(df["dl-CarrierFreq-r10"].iloc[i]).split('@')
            orig_scells = scells
            scells = [(int(float(pci)), int(float(freq))) for pci, freq in zip(pcis, freqs)]

            others += f' Set up {len(scells)} SCell.'
            trans = f'{orig_scells} -> {scells}'

            end, _ = find_1st_after(i,'rrcConnectionReconfigurationComplete')
            
            a, _ = find_1st_before(i, "rrcConnectionReestablishmentRequest", 3)
            if a is not None:
                others += ' Near after RLF.'

            a = find_in_D_exact(['LTE_HO', 'MN_HO', 'MN_HO_to_eNB', 'SN_setup', 'SN_Rel'])
            if a is not None:
                others += f' With {a}.'

            D['Add_SCell'].append(HO(start=t,end=end,others=others, trans=trans))
    
    return D

##### Modified Version from Sheng-Ru
def parse_trans(item):
    chunk = item.split(' | ')
    # print(chunk)
    if len(chunk) == 1:
        s_src = None
        s_tgt = None
        if chunk[0] == '':
            m_src = None
            m_tgt = None
        elif chunk[0][0] == '?':
            m_src = None
            m_tgt = chunk[0].split(' -> ')[1]
        else:
            m_src = chunk[0].split(' -> ')[0]
            m_tgt = chunk[0].split(' -> ')[1]
    else:
        if chunk[1] == 'O':
            s_src = None
            s_tgt = None
        else:
            chunk1 = chunk[1].split(' -> ')
            if len(chunk1) == 1:
                s_src = chunk1[0]
                s_tgt = None
            else:
                s_src = chunk1[0] if chunk1[0] != 'O' else None
                s_tgt = chunk1[1] if chunk1[1] != 'O' else None
            
        chunk1 = chunk[0].split(' -> ')
        if len(chunk1) == 1:
            m_src = chunk1[0]
            m_tgt = None
        else:
            m_src = chunk1[0]
            m_tgt = chunk1[1]
            
    return m_src, m_tgt, s_src, s_tgt

def parse_handover(df, tz=8):
    
    key_mapping = {
        'Conn_Rel': 'Conn_REL',
        'Conn_Req': 'Conn_SU',
        'LTE_HO': 'LTE_HO',
        'MN_HO': 'MN_HO',
        'MN_HO_to_eNB': 'MN_HO_SN_REL',
        'SN_setup': 'SN_SU',
        'SN_Rel': 'SN_REL',
        'SN_HO': 'SN_HO',
        'RLF_II': 'MCG_FAIL',
        'RLF_III': 'NAS_RECOV',
        'SCG_RLF': 'SCG_FAIL',
        'Add_SCell': 'Add_SCell'
    }
    
    D = parse_mi_ho(df, tz)
    
    new_D = {key_mapping.get(key, key): value for key, value in D.items()}
    table = pd.DataFrame()
    for key, lst in new_D.items():
        table1 = pd.DataFrame(lst, index=[key]*len(lst)).reset_index(names='type')
        table = pd.concat([table, table1], ignore_index=True)

    sc_info = df[df['type_id'] == 'LTE_RRC_Serv_Cell_Info'][['Timestamp', 'type_id', 'PCI', 'Cell Identity', 'Band ID', 'DL frequency', 'UL frequency', 'DL bandwidth', 'UL bandwidth', 'TAC', 'MCC', 'MNC']].reset_index(drop=True).rename(columns={'Timestamp': 'start', 'type_id': 'type'})
    sc_info['eNB_ID'] = sc_info['Cell Identity'] // 256
    sc_info['Cell_ID'] = sc_info['Cell Identity'] % 256
    sc_info = sc_info[['start', 'type', 'PCI', 'eNB_ID', 'Cell_ID', 'Cell Identity', 'Band ID', 'DL frequency', 'UL frequency', 'DL bandwidth', 'UL bandwidth', 'TAC', 'MCC', 'MNC']]

    table = pd.concat([table, sc_info], ignore_index=True).sort_values(by='start').reset_index(drop=True)

    is_not_start = True
    selected_cols = ['PCI', 'eNB_ID', 'Cell_ID', 'Cell Identity', 'Band ID', 'DL frequency', 'UL frequency', 'DL bandwidth', 'UL bandwidth', 'TAC', 'MCC', 'MNC']
    for i, row in table.iterrows():
        if row['type'] == 'LTE_RRC_Serv_Cell_Info':
            is_not_start = False
            info_to_fill = row[selected_cols].to_list()
            continue
        if is_not_start:
            continue
        table.loc[i, selected_cols] = info_to_fill

    table = table[table['type'] != 'LTE_RRC_Serv_Cell_Info'].reset_index(drop=True)
    
    for i, row in table.iterrows():
        table.loc[i, ['m_src', 'm_tgt', 's_src', 's_tgt']] = parse_trans(row['trans'])
    
    table1 = table[np.in1d(table['type'], ['Add_SCell'])]
    table = table[~np.in1d(table['type'], ['Add_SCell'])].reset_index(drop=True)
    
    table['next_eNB'] = table['eNB_ID'].shift(-1)
    for i, row in table.iloc[:-1].iterrows():
        if row['eNB_ID'] != row['next_eNB']:
            if row['others'] == '':
                table.loc[i, 'others'] = 'Inter eNB HO.'
            else:
                table.loc[i, 'others'] += ' Inter eNB HO.'
    
    table = pd.concat([table, table1], ignore_index=True).sort_values(by='start').reset_index(drop=True)
    
    table1 = table[~np.in1d(table['type'], ['SN_SU', 'SN_REL'])]
    table = table[np.in1d(table['type'], ['SN_SU', 'SN_REL'])].reset_index(drop=True)
    
    table['prev_cmt'] = table['others'].shift(1)
    for i, row in table.iloc[1:].iterrows():
        if row['type'] == 'SN_SU' and 'Caused by scg-failure' in row['prev_cmt']:
            if 'Near after SN_Rel' in row['others']:
                table.loc[i, 'others'] += ' Caused by scg-failure.'
    
    table = pd.concat([table, table1], ignore_index=True).sort_values(by='start').reset_index(drop=True)
    table = table[['type', 'start', 'end', 'others', 'm_src', 'm_tgt', 's_src', 's_tgt', *selected_cols]]
    
    return table, new_D

In [4]:
data_folder = "/Users/jackbedford/Desktop/MOXA/Code/data/2023-09-12_2/Bandlock_9_Schemes_Phone_UDP"
setting = {'sm00': 'All', 'sm01': 'All', 'sm02': 'B3', 'sm03': 'B7', 'sm04': 'B8', 'sm05': 'B3B7', 'sm06': 'B3B8', 'sm07': 'B7B8', 'sm08': 'LTE'}
devices = list(setting.keys())
schemes = list(setting.values())
trips = 2

rrc_files = {}
ho_dfs = {}
for dev in devices:
    rrc_files[dev] = []
    ho_dfs[dev] = []
    for tr in [f"#{i+1:02d}" for i in range(trips)]:
        rrc_file = [os.path.join(data_folder, dev, tr, "data", s) for s in os.listdir(os.path.join(data_folder, dev, tr, "data")) if s.startswith("diag") and s.endswith("rrc.pkl")][0]
        rrc_files[dev].append(rrc_file)
        # TODO: Do anything you want!

# Start Analysis

In [5]:
devices = ['sm06', 'sm05', 'sm07']
schemes = ['B3B8', 'B3B7', 'B7B8']
tr = '#01'

root = '/Users/jackbedford/Desktop/MOXA/Code/data/2023-09-12_2/Bandlock_9_Schemes_Phone_UDP'

rrc_files = [[os.path.join(root, dev, tr, 'data', s) for s in os.listdir(os.path.join(root, dev, tr, 'data')) if s.startswith('diag') and s.endswith('rrc.pkl')][0] for dev in devices]
nr_files = [[os.path.join(root, dev, tr, 'data', s) for s in os.listdir(os.path.join(root, dev, tr, 'data')) if s.startswith('diag') and s.endswith('nr_ml1.pkl')][0] for dev in devices]
lte_files = [[os.path.join(root, dev, tr, 'data', s) for s in os.listdir(os.path.join(root, dev, tr, 'data')) if s.startswith('diag') and s.endswith('ml1.pkl') and not s.endswith('nr_ml1.pkl')][0] for dev in devices]
dl_files = [os.path.join(root, dev, tr, 'data', 'udp_dnlk_loss_latency.pkl') for dev in devices]

pprint(rrc_files)
pprint(nr_files)
pprint(lte_files)
pprint(dl_files)

['/Users/jackbedford/Desktop/MOXA/Code/data/2023-09-12_2/Bandlock_9_Schemes_Phone_UDP/sm06/#01/data/diag_log_sm06_2023-09-12_13-34-15_rrc.pkl',
 '/Users/jackbedford/Desktop/MOXA/Code/data/2023-09-12_2/Bandlock_9_Schemes_Phone_UDP/sm05/#01/data/diag_log_sm05_2023-09-12_13-34-15_rrc.pkl',
 '/Users/jackbedford/Desktop/MOXA/Code/data/2023-09-12_2/Bandlock_9_Schemes_Phone_UDP/sm07/#01/data/diag_log_sm07_2023-09-12_13-34-15_rrc.pkl']
['/Users/jackbedford/Desktop/MOXA/Code/data/2023-09-12_2/Bandlock_9_Schemes_Phone_UDP/sm06/#01/data/diag_log_sm06_2023-09-12_13-34-15_nr_ml1.pkl',
 '/Users/jackbedford/Desktop/MOXA/Code/data/2023-09-12_2/Bandlock_9_Schemes_Phone_UDP/sm05/#01/data/diag_log_sm05_2023-09-12_13-34-15_nr_ml1.pkl',
 '/Users/jackbedford/Desktop/MOXA/Code/data/2023-09-12_2/Bandlock_9_Schemes_Phone_UDP/sm07/#01/data/diag_log_sm07_2023-09-12_13-34-15_nr_ml1.pkl']
['/Users/jackbedford/Desktop/MOXA/Code/data/2023-09-12_2/Bandlock_9_Schemes_Phone_UDP/sm06/#01/data/diag_log_sm06_2023-09-12_13

# Handover Analysis

In [6]:
ImpactScope = {
    'LTE_HO': (2, 1),
    'MN_HO': (3, 2.5),
    'MN_HO_SN_REL': (1, 1),
    'SN_HO': (6, 6),
    'SN_SU': (1, 2),
    'SN_REL': (0.5, 0.5),
    'SCG_FAIL': (2.5, 1.5),
    'MCG_FAIL': (6, 2),
    'NAS_RECOV': (4, 2),
}
sorter = ['LTE_HO', 'MN_HO', 'MN_HO_SN_REL', 'SN_HO', 'SN_SU', 'SN_REL', 'SCG_FAIL', 'MCG_FAIL', 'NAS_RECOV']

def cut_head_tail(df_HO, df, mode='ul'):
    if mode == 'ul':
        start = df.iloc[0].xmit_time
        stop = df.iloc[-1].xmit_time
        df_HO = df_HO.query('start >= @start & start <= @stop').copy().reset_index(drop=True)
    if mode == 'dl':
        start = df.iloc[0].arr_time
        stop = df.iloc[-1].arr_time
        df_HO = df_HO.query('start >= @start & start <= @stop').copy().reset_index(drop=True)
    return df_HO

def is_disjoint(set1, set2):
    """
    Check if two sets are disjoint.
    """
    return (set1 & set2).empty

def is_disjoint_dict(E):
    test_intv = P.empty()
    for key, val in E.items():
        # print(key)
        for intv in val:
            if is_disjoint(test_intv, intv.interval):
                test_intv = test_intv | intv.interval
            else:
                print(key, intv.index)
                return False
    return True

def interp(x, y, ratio):
    """
    Interpolation

    Args:
        x, y (datetime.datetime): x < y
        ratio (float): a decimal numeral in a range [0, 1]; 0 means break at x, 1 means break at y.
    Returns:
        (datetime.datetime): breakpoint of interpolation
    """
    return x + (y - x) * ratio

def get_ho_interval(df, sec=(1, 3), ratio=0.5,
                 ignored=['Conn_Setup','Conn_Rel'],
                 handover=['LTE_HO', 'MN_HO', 'MN_HO_SN_REL', 'SN_HO', 'SN_SU', 'SN_REL'],
                 linkfailure=['SCG_FAIL', 'MCG_FAIL', 'NAS_RECOV'],
                 ImpactScope=ImpactScope):
    
    HO_INTV = namedtuple('HO_INTV', 'index, interval, m_src, m_tgt, s_src, s_tgt, PCI, eNB_ID, Cell_ID, Band_ID', defaults=tuple([None]*10))
    
    def ignore_col(row):
        if row.type in ignored:
            return False
        else:
            return True
    df = df[df.apply(ignore_col, axis=1)].reset_index(drop=True)
    
    column_names = []
    for type_name in handover + linkfailure:
        column_names += ["before_{}".format(type_name), "during_{}".format(type_name), "after_{}".format(type_name)]
    E = { col:[] for col in column_names }
    
    for i, row in df.iterrows():
        prior_row = df.iloc[i-1] if i != 0 else None
        post_row = df.iloc[i+1] if i != len(df)-1 else None
        ### peek the next event
        if i != len(df)-1 and pd.notna(row.end) and row.end > post_row.start:
            # print(i, row.start, row.end, row.type, row.cause)
            # print(i+1, post_row.start, post_row.end, post_row.type, post_row.cause)
            print(i, row.start, row.end, row.type)
            print(i+1, post_row.start, post_row.end, post_row.type)
            continue
        ### peri_interval
        if pd.isna(row.end):
            peri_interval = P.singleton(row.start)
        else:
            peri_interval = P.closed(row.start, row.end)
        ### prior_interval
        type_name = row.type if row.type not in ['SCG_FAIL', 'MCG_FAIL', 'NAS_RECOV'] else row.type # + '_' + row.cause
        C = row.start - pd.Timedelta(seconds=ImpactScope[type_name][0])
        D = row.start
        prior_interval = P.closedopen(C, D)
        if ratio != None and i != 0:
            type_name = prior_row.type if prior_row.type not in ['SCG_FAIL', 'MCG_FAIL', 'NAS_RECOV'] else prior_row.type # + '_' + prior_row.cause
            A = max(prior_row.start, prior_row.end)
            B = max(prior_row.start, prior_row.end) + pd.Timedelta(seconds=ImpactScope[type_name][1])
            if P.openclosed(A, B).overlaps(prior_interval):
                # print("Overlaps with the previous!")
                bkp = interp(C, B, ratio)
                bkp = max(bkp, A)  # to avoid the breakpoint overlaps the previous event's duration
                # bkp = min(max(bkp, A), D)  # 我不侵犯到其他任何人，代表其他人也不會侵犯到我！
                prior_interval = P.closedopen(bkp, D)
                if A in prior_interval:
                    prior_interval = P.open(bkp, D)
                # blindly set as open inverval is fine, but may miss one point.
        ### post_interval
        type_name = row.type if row.type not in ['SCG_FAIL', 'MCG_FAIL', 'NAS_RECOV'] else row.type # + '_' + row.cause
        C = row.end
        D = row.end + pd.Timedelta(seconds=ImpactScope[type_name][1])
        post_interval = P.openclosed(C, D)
        if ratio != None and i != len(df)-1:
            type_name = post_row.type if post_row.type not in ['SCG_FAIL', 'MCG_FAIL', 'NAS_RECOV'] else post_row.type # + '_' + post_row.cause
            A = min(post_row.start, post_row.end) - pd.Timedelta(seconds=ImpactScope[type_name][0])
            B = min(post_row.start, post_row.end)
            if P.closedopen(A, B).overlaps(post_interval):
                # print("Overlaps with the following!")
                bkp = interp(A, D, ratio)
                bkp = min(bkp, B)  # to avoid the breakpoint overlaps the following event's duration
                # bkp = max(min(bkp, B), C)  # 我不侵犯到其他任何人，代表其他人也不會侵犯到我！
                post_interval = P.open(C, bkp)
        ### append dictionary
        type_name = row.type if row.type not in ['SCG_FAIL', 'MCG_FAIL', 'NAS_RECOV'] else row.type # + '_' + row.cause
        # state1, state2 = 'sn_change', 'sn_change'
        # if type_name in linkfailure:
        #     state1, state2 = 'link_failure', 'link_failure'
        # if type_name in ['LTE_HO','MN_HO','SN_Rel_MN_HO']:
        #     state1 = 'inter_freq' if row.sFreq != row.tFreq else 'intra_freq'
        #     if pd.notna(row.eNB) and pd.notna(row.eNB1) and row.eNB != row.eNB1:
        #         state2 = 'inter_enb'
        #     elif row.sPCI != row.tPCI:
        #         state2 = 'inter_sector'
        #     elif row.sPCI == row.tPCI:
        #         state2 = 'intra_sector'
        #     else:
        #         print("************** inter_enb, unknown eNB_ID **************")
        #         state2 = 'inter_enb'
        E[f'before_{type_name}'].append(HO_INTV(i, prior_interval, row.m_src, row.m_tgt, row.s_src, row.s_tgt, row.PCI, row.eNB_ID, row.Cell_ID, row['Band ID']))
        E[f'during_{type_name}'].append(HO_INTV(i, peri_interval, row.m_src, row.m_tgt, row.s_src, row.s_tgt, row.PCI, row.eNB_ID, row.Cell_ID, row['Band ID']))
        E[f'after_{type_name}'].append(HO_INTV(i, post_interval, row.m_src, row.m_tgt, row.s_src, row.s_tgt, row.PCI, row.eNB_ID, row.Cell_ID, row['Band ID']))
        ### check whether the intervals are pairwise disjoint
        if not is_disjoint_dict(E):
            print('Warning: Intervals are not totally disjoint!')
    return E

def label_ho_info(df, E, mode='ul'):
    def removeprefix(string, prefix=['before','during','after']):
        for pref in prefix:
            if string.startswith(pref):
                return pref, string[len(pref)+1:]
        return None, string
    
    df = df.reindex(columns=[*list(df.columns),
            'ho_index','ho_stage','ho_type','ho_m_src', 'ho_m_tgt', 'ho_s_src', 'ho_s_tgt', 'ho_PCI', 'ho_eNB_ID', 'ho_Cell_ID', 'ho_Band_ID'])
    
    df[['ho_index', 'ho_stage', 'ho_type', 'ho_m_src', 'ho_m_tgt', 'ho_s_src', 'ho_s_tgt', 'ho_PCI', 'ho_eNB_ID', 'ho_Cell_ID', 'ho_Band_ID']] = \
        [-1, '-', 'stable',  None, None, None, None, None, None, None, None]
            
    for key, val in E.items():
        pref, key = removeprefix(key)
        for intv in val:
            if intv.interval.empty:
                continue
            # print(pref, key)
            # print(intv.interval)
            if mode == 'ul':
                df.loc[(df['xmit_time'] >= intv.interval.lower) & (df['xmit_time'] <= intv.interval.upper),
                       ('ho_index', 'ho_stage', 'ho_type', 'ho_m_src', 'ho_m_tgt', 'ho_s_src', 'ho_s_tgt', 'ho_PCI', 'ho_eNB_ID', 'ho_Cell_ID', 'ho_Band_ID')] = \
                        [intv.index, pref, key, intv.m_src, intv.m_tgt, intv.s_src, intv.s_tgt, intv.PCI, intv.eNB_ID, intv.Cell_ID, intv.Band_ID]
            if mode == 'dl':
                df.loc[(df['arr_time'] >= intv.interval.lower) & (df['arr_time'] <= intv.interval.upper),
                       ('ho_index', 'ho_stage', 'ho_type', 'ho_m_src', 'ho_m_tgt', 'ho_s_src', 'ho_s_tgt', 'ho_PCI', 'ho_eNB_ID', 'ho_Cell_ID', 'ho_Band_ID')] = \
                        [intv.index, pref, key, intv.m_src, intv.m_tgt, intv.s_src, intv.s_tgt, intv.PCI, intv.eNB_ID, intv.Cell_ID, intv.Band_ID]
    
    # df['ho_type0'] = df['ho_type']
    # df.loc[np.in1d(df['ho_type'], ['SCG_Failure','MCG_Failure','NAS_Recovery']), 'ho_type0'] = \
    #     df.loc[np.in1d(df['ho_type'], ['SCG_Failure','MCG_Failure','NAS_Recovery']), 'ho_type'] + '_' + df.loc[np.in1d(df['ho_type'], ['SCG_Failure','MCG_Failure','NAS_Recovery']), 'ho_cause']
    
    # df['_ho_type'] = df['ho_type']
    # df['_ho_type0'] = df['ho_type0']
    # df['_ho_type1'] = df['ho_type1']
    # df['_ho_type2'] = df['ho_type2']
    # df.loc[~np.in1d(df['ho_type'], ['stable']), '_ho_type'] = df.loc[~np.in1d(df['ho_type'], ['stable']), 'ho_stage'] + '_' + df.loc[~np.in1d(df['ho_type'], ['stable']), 'ho_type']
    # df.loc[~np.in1d(df['ho_type'], ['stable']), '_ho_type0'] = df.loc[~np.in1d(df['ho_type'], ['stable']), 'ho_stage'] + '_' + df.loc[~np.in1d(df['ho_type'], ['stable']), 'ho_type0']
    # df.loc[~np.in1d(df['ho_type'], ['stable']), '_ho_type1'] = df.loc[~np.in1d(df['ho_type'], ['stable']), 'ho_stage'] + '_' + df.loc[~np.in1d(df['ho_type'], ['stable']), 'ho_type1']
    # df.loc[~np.in1d(df['ho_type'], ['stable']), '_ho_type2'] = df.loc[~np.in1d(df['ho_type'], ['stable']), 'ho_stage'] + '_' + df.loc[~np.in1d(df['ho_type'], ['stable']), 'ho_type2']
    
    # df['ho_index'] = df['ho_index'].astype('Int32')
    # df['ho_stage'] = df['ho_stage'].astype('category')
    # df['ho_type'] = df['ho_type'].astype('category')
    # df['ho_type0'] = df['ho_type0'].astype('category')
    # df['ho_type1'] = df['ho_type1'].astype('category')
    # df['ho_type2'] = df['ho_type2'].astype('category')
    # df['ho_scel'] = df['ho_scel'].astype('Int8')
    # df['ho_cause'] = df['ho_cause'].astype('category')
    # df['ho_intr'] = df['ho_intr'].astype('float32')
    # for tag in df.columns[df.columns.get_loc('ho_ePCI'):df.columns.get_loc('ho_nrPCI1')+1]:
    #     df[tag] = df[tag].astype('Int32')
    # df['_ho_type'] = df['_ho_type'].astype('category')
    # df['_ho_type0'] = df['_ho_type'].astype('category')
    # df['_ho_type1'] = df['_ho_type1'].astype('category')
    # df['_ho_type2'] = df['_ho_type2'].astype('category')
    
    return df

In [7]:
ho_dfs = []

for filepath in rrc_files:
    df = pd.read_pickle(filepath)
    table, _ = parse_handover(df)
    table = table[~np.in1d(table['type'], ['Add_SCell', 'Conn_REL', 'Conn_SU'])].reset_index(drop=True)
    ho_dfs.append(table)

In [8]:
dl_dfs = []

for filepath in dl_files:
    df = pd.read_pickle(filepath)
    df = set_data(df)
    df = df[['seq', 'Timestamp', 'lost', 'excl', 'latency', 'xmit_time', 'arr_time']]
    dl_dfs.append(df)

In [42]:
pci_lists = []
for df in ho_dfs:
    df = df[np.in1d(df['type'], ['MN_HO', 'MN_HO_SN_REL', 'LTE_HO', 'MCG_FAIL', 'NAS_RECOV'])].reset_index(drop=True)
    pci_lists.append([(ast.literal_eval(s1), int(s2)) for s1, s2 in zip(df['m_src'].to_list(), df['Band ID'].to_list())])

for item in pci_lists:
    print(item)

[((362, 1750), 3), ((73, 1750), 3), ((378, 1750), 3), ((73, 1750), 3), ((294, 1750), 3), ((73, 1750), 3), ((16, 1750), 3), ((2, 1400), 3), ((180, 1750), 3), ((188, 1750), 3), ((197, 1750), 3), ((205, 1750), 3), ((11, 1750), 3), ((55, 1750), 3), ((130, 1750), 3), ((122, 1750), 3), ((386, 1750), 3), ((122, 1750), 3), ((386, 1750), 3), ((186, 1750), 3), ((400, 1750), 3), ((27, 1750), 3), ((152, 1750), 3), ((27, 1750), 3), ((43, 1750), 3), ((400, 1750), 3), ((392, 1750), 3), ((382, 1750), 3), ((43, 1750), 3), ((382, 1750), 3), ((75, 1750), 3), ((117, 1750), 3), ((75, 1750), 3), ((376, 1750), 3), ((360, 1750), 3), ((75, 1750), 3), ((83, 1750), 3), ((67, 1750), 3), ((140, 1750), 3), ((315, 1750), 3), ((331, 1750), 3), ((118, 1750), 3), ((7, 1750), 3), ((15, 1750), 3), ((319, 1750), 3), ((240, 1750), 3), ((36, 1750), 3), ((240, 1750), 3), ((36, 1750), 3), ((44, 1750), 3), ((329, 1750), 3), ((87, 1750), 3), ((287, 1750), 3), ((271, 1750), 3), ((69, 1750), 3), ((84, 1750), 3), ((353, 1750), 3),

In [10]:
E_list = []

for i in range(3):
    ho_dfs[i] = cut_head_tail(ho_dfs[i], dl_dfs[i], mode='dl')
    E_list.append(get_ho_interval(ho_dfs[i], ImpactScope=ImpactScope))
    dl_dfs[i] = label_ho_info(dl_dfs[i], E_list[i], mode='dl')

195 2023-09-12 13:57:58.365644 2023-09-12 13:57:58.454081 SCG_FAIL
196 2023-09-12 13:57:58.433164 2023-09-12 13:57:58.454081 SN_REL
8 2023-09-12 13:35:12.949245 2023-09-12 13:35:13.027818 SCG_FAIL
9 2023-09-12 13:35:12.992957 2023-09-12 13:35:13.027818 SN_REL
before_SN_REL 9
before_SN_REL 9
before_SN_REL 9
before_SN_REL 9
before_SN_REL 9
before_SN_REL 9
before_SN_REL 9
before_SN_REL 9
before_SN_REL 9
before_SN_REL 9
before_SN_REL 9
before_SN_REL 9
before_SN_REL 9
before_SN_REL 9
before_SN_REL 9
before_SN_REL 9
before_SN_REL 9
before_SN_REL 9
before_SN_REL 9
before_SN_REL 9
before_SN_REL 9
before_SN_REL 9
before_SN_REL 9
before_SN_REL 9
before_SN_REL 9
before_SN_REL 9
before_SN_REL 9
before_SN_REL 9
before_SN_REL 9
before_SN_REL 9
before_SN_REL 9
before_SN_REL 9
before_SN_REL 9
before_SN_REL 9
before_SN_REL 9
before_SN_REL 9
before_SN_REL 9
before_SN_REL 9
before_SN_REL 9
before_SN_REL 9
before_SN_REL 9
before_SN_REL 9
before_SN_REL 9
before_SN_REL 9
before_SN_REL 9
before_SN_REL 9
befo

In [11]:
dl_dfs[0][dl_dfs[0]['ho_type'] != 'stable']

,seq,Timestamp,lost,excl,latency,xmit_time,arr_time,ho_index,ho_stage,ho_type,ho_m_src,ho_m_tgt,ho_s_src,ho_s_tgt,ho_PCI,ho_eNB_ID,ho_Cell_ID,ho_Band_ID
1229,3730,2023-09-12 13:34:18.703740,False,False,0.011114,2023-09-12 13:34:18.703765,2023-09-12 13:34:18.714854220,0,before,SN_HO,"(362, 1750)",None,350,362,362.0,212939.0,23.0,3.0
1230,3731,2023-09-12 13:34:18.705752,False,False,0.009102,2023-09-12 13:34:18.705796,2023-09-12 13:34:18.714854220,0,before,SN_HO,"(362, 1750)",None,350,362,362.0,212939.0,23.0,3.0
1231,3732,2023-09-12 13:34:18.707740,False,False,0.007114,2023-09-12 13:34:18.707767,2023-09-12 13:34:18.714854220,0,before,SN_HO,"(362, 1750)",None,350,362,362.0,212939.0,23.0,3.0
1232,3733,2023-09-12 13:34:18.709740,False,False,0.010908,2023-09-12 13:34:18.709767,2023-09-12 13:34:18.720648220,0,before,SN_HO,"(362, 1750)",None,350,362,362.0,212939.0,23.0,3.0
1233,3734,2023-09-12 13:34:18.711740,False,False,0.008908,2023-09-12 13:34:18.711781,2023-09-12 13:34:18.720648220,0,before,SN_HO,"(362, 1750)",None,350,362,362.0,212939.0,23.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1400173,1402674,2023-09-12 14:20:56.722486,False,False,0.004221,2023-09-12 14:20:56.722514,2023-09-12 14:20:56.726707220,294,after,LTE_HO,"(349, 1400)","(357, 1400)",None,None,349.0,219266.0,32.0,3.0
1400174,1402675,2023-09-12 14:20:56.724486,False,False,0.002221,2023-09-12 14:20:56.724511,2023-09-12 14:20:56.726707220,294,after,LTE_HO,"(349, 1400)","(357, 1400)",None,None,349.0,219266.0,32.0,3.0
1400175,1402676,2023-09-12 14:20:56.726486,False,False,0.000221,2023-09-12 14:20:56.726513,2023-09-12 14:20:56.726707220,294,after,LTE_HO,"(349, 1400)","(357, 1400)",None,None,349.0,219266.0,32.0,3.0
1400176,1402677,2023-09-12 14:20:56.728486,False,False,-0.001779,2023-09-12 14:20:56.728514,2023-09-12 14:20:56.726707220,294,after,LTE_HO,"(349, 1400)","(357, 1400)",None,None,349.0,219266.0,32.0,3.0


In [12]:
sorter = ['LTE_HO', 'MN_HO', 'MN_HO_SN_REL', 'SN_HO', 'SN_SU', 'SN_REL', 'SCG_FAIL', 'MCG_FAIL', 'NAS_RECOV']

for tag in sorter:
    print(tag, end=', ')
print()
print('-----------------------------------')

for df in ho_dfs:
    for tag in sorter:
        print(sum(df['type'] == tag), end=', ')
    print()
print('-----------------------------------')

for df in dl_dfs:
    for tag in sorter:
        print(sum((df['ho_type'] == tag) & (df['lost'] == True)), end=', ')
    print()

LTE_HO, MN_HO, MN_HO_SN_REL, SN_HO, SN_SU, SN_REL, SCG_FAIL, MCG_FAIL, NAS_RECOV, 
-----------------------------------
97, 82, 2, 80, 14, 4, 1, 15, 0, 
98, 79, 6, 66, 23, 8, 4, 11, 0, 
113, 65, 7, 41, 17, 4, 2, 8, 0, 
-----------------------------------
1121, 1858, 0, 1323, 1691, 0, 0, 5741, 0, 
115, 43, 0, 1439, 2376, 144, 0, 7610, 0, 
833, 453, 152, 693, 757, 0, 0, 585, 0, 


In [14]:
for df in dl_dfs:
    for tag in sorter:
        print(round(sum((df['ho_type'] == tag) & (df['lost'] == True)) / (sum((df['ho_type'] == tag) & (df['lost'] == False)) + 1e-9) * 100, 2), end=', ')
    print()

0.92, 1.78, 0.0, 0.53, 22.47, 0.0, 0.0, 25.66, 0.0, 
0.09, 0.04, 0.0, 0.71, 14.86, 6.92, 0.0, 51.65, 0.0, 
0.57, 0.5, 14.67, 0.52, 6.28, 0.0, 0.0, 4.16, 0.0, 


# Dual

In [16]:
dual_dl_dfs = []

dual_dl_dfs.append(pd.merge(dl_dfs[0].copy(), dl_dfs[1].copy(), on=['seq'], suffixes=('_m','_s')).copy())
dual_dl_dfs.append(pd.merge(dl_dfs[0].copy(), dl_dfs[2].copy(), on=['seq'], suffixes=('_m','_s')).copy())

In [18]:
for i in range(2):
    dual_dl_dfs[i]['lost_ms'] = dual_dl_dfs[i]['lost_m'] & dual_dl_dfs[i]['lost_s']

In [21]:
for df in dual_dl_dfs:
    for tag in sorter:
        print(sum((df['ho_type_m'] == tag) & (df['lost_ms'] == True)), end=', ')
    print()
    for tag in sorter:
        print(round(sum((df['ho_type_m'] == tag) & (df['lost_ms'] == True)) / (sum((df['ho_type_m'] == tag) & (df['lost_ms'] == False)) + 1e-9) * 100, 2), end=', ')
    print()

0, 629, 0, 650, 398, 0, 0, 1319, 0, 
0.0, 0.6, 0.0, 0.26, 4.51, 0.0, 0.0, 4.92, 0.0, 
0, 0, 0, 4, 0, 0, 0, 1, 0, 
0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 


In [23]:
for df in dual_dl_dfs:
    for tag in [*sorter, 'stable']:
        print(sum((df['ho_type_m'] == 'MCG_FAIL') & (df['ho_type_s'] == tag) & (df['lost_ms'] == True)), end=', ')
    print()

for df in dual_dl_dfs:
    for tag in [*sorter, 'stable']:
        print(sum((df['ho_type_m'] == 'SN_SU') & (df['ho_type_s'] == tag) & (df['lost_ms'] == True)), end=', ')
    print()

for df in dual_dl_dfs:
    for tag in [*sorter, 'stable']:
        print(sum((df['ho_type_m'] == 'SN_HO') & (df['ho_type_s'] == tag) & (df['lost_ms'] == True)), end=', ')
    print()

0, 0, 0, 0, 18, 0, 0, 1301, 0, 0, 
0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 
0, 0, 0, 0, 380, 0, 0, 18, 0, 0, 
0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 
0, 0, 0, 104, 0, 0, 0, 546, 0, 0, 
0, 0, 0, 4, 0, 0, 0, 0, 0, 0, 


In [ ]:
def get_intervals_length(intervals):
    """
    Get total length of a set of intervals.
    """
    if intervals.empty:
        return 0
    sum = 0
    for s in intervals:
        sum += (s.upper - s.lower) / dt.timedelta(seconds=1)
    return round(sum, 6)

In [39]:
m_radio = []

for tag in ['MCG_FAIL', 'SN_SU', 'SN_HO']:
    tmp_interval = P.empty()
    for prefix in ['before_', 'during_', 'after_']:
        for item in E_list[0][prefix+tag]:
            tmp_interval = tmp_interval | item.interval
    m_radio.append(tmp_interval)

s_radios = [[], []]
for i in range(2):
    for tag in sorter:
        tmp_interval = P.empty()
        for prefix in ['before_', 'during_', 'after_']:
            for item in E_list[i+1][prefix+tag]:
                tmp_interval = tmp_interval | item.interval
        s_radios[i].append(tmp_interval)

print(m_radio)
print(s_radios)

[[Timestamp('2023-09-12 13:35:14.084905500'),Timestamp('2023-09-12 13:35:18.570154')) | [Timestamp('2023-09-12 13:36:23.017635500'),Timestamp('2023-09-12 13:36:24.254478')) | [Timestamp('2023-09-12 13:38:28.633084'),Timestamp('2023-09-12 13:38:31.128315')) | [Timestamp('2023-09-12 13:44:48.883076'),Timestamp('2023-09-12 13:44:51.947840')) | [Timestamp('2023-09-12 13:50:44.973959500'),Timestamp('2023-09-12 13:50:47.705820')) | [Timestamp('2023-09-12 13:51:03.242430500'),Timestamp('2023-09-12 13:51:07.148114')) | [Timestamp('2023-09-12 13:52:30.216589'),Timestamp('2023-09-12 13:52:34.109146')) | (Timestamp('2023-09-12 13:52:34.148349'),Timestamp('2023-09-12 13:52:35.926877')) | (Timestamp('2023-09-12 13:55:05.097772'),Timestamp('2023-09-12 13:55:08.885557')) | (Timestamp('2023-09-12 14:06:03.414679'),Timestamp('2023-09-12 14:06:08.359429')] | (Timestamp('2023-09-12 14:07:31.319370'),Timestamp('2023-09-12 14:07:35.926875')] | (Timestamp('2023-09-12 14:07:47.275507'),Timestamp('2023-09-12 

In [41]:
for item in m_radio:
    for s_radio in s_radios:
        for item1 in s_radio:
            print(round(get_intervals_length(item & item1), 3), end=', ')
        print()
        for item1 in s_radio:
            print(round(get_intervals_length(item & item1) / (get_intervals_length(item) + 1e-9)*100, 2), end=', ')
        print()

7.287, 3.304, 0.313, 8.524, 2.927, 0, 0, 17.116, 0, 
12.86, 5.83, 0.55, 15.04, 5.17, 0.0, 0.0, 30.2, 0.0, 
15.694, 4.462, 0, 10.962, 0, 0, 0, 2.495, 0, 
27.69, 7.87, 0.0, 19.34, 0.0, 0.0, 0.0, 4.4, 0.0, 
1.068, 0, 0, 2.714, 8.084, 0.545, 0, 2.39, 0, 
5.82, 0.0, 0.0, 14.79, 44.04, 2.97, 0.0, 13.02, 0.0, 
0.465, 2.472, 0.061, 4.773, 2.174, 0, 0, 1.81, 0, 
2.53, 13.47, 0.33, 26.0, 11.85, 0.0, 0.0, 9.86, 0.0, 
0.267, 57.398, 2.346, 292.907, 14.869, 1.62, 0, 15.637, 0, 
0.05, 11.53, 0.47, 58.82, 2.99, 0.33, 0.0, 3.14, 0.0, 
45.509, 50.873, 1.239, 193.35, 8.237, 0.635, 0, 8.782, 0, 
9.14, 10.22, 0.25, 38.83, 1.65, 0.13, 0.0, 1.76, 0.0, 
